
# Advanced Time Series Forecasting with Deep Learning & Attention (Colab)

This notebook includes:
- Synthetic dataset generation
- Sliding window preprocessing
- Transformer-based forecasting model
- Evaluation using RMSE & MAE
- ARIMA baseline comparison


In [ ]:

!pip install torch numpy pandas scikit-learn matplotlib statsmodels


## Dataset Generation

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(42)
T = 3000
t = np.arange(T)

trend = 0.005 * t
seasonal1 = 2 * np.sin(2 * np.pi * t / 50)
seasonal2 = 1.5 * np.sin(2 * np.pi * t / 365)
long_dep = 0.8 * np.sin(2 * np.pi * t / 800)
noise = np.random.normal(0, 0.5, T)

series = 10 + trend + seasonal1 + seasonal2 + long_dep + noise
df = pd.DataFrame({"value": series})
df.head()


In [ ]:

plt.figure(figsize=(12,4))
plt.plot(df['value'])
plt.title("Synthetic Time Series")
plt.show()


## Data Pipeline

In [ ]:

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

SEQ_LEN = 60
PRED_LEN = 1
BATCH_SIZE = 64

class TimeSeriesDataset(Dataset):
    def __init__(self, values):
        self.scaler = StandardScaler()
        self.values = self.scaler.fit_transform(values.reshape(-1,1))

    def __len__(self):
        return len(self.values) - SEQ_LEN - PRED_LEN

    def __getitem__(self, idx):
        x = self.values[idx:idx+SEQ_LEN]
        y = self.values[idx+SEQ_LEN:idx+SEQ_LEN+PRED_LEN]
        return torch.FloatTensor(x), torch.FloatTensor(y)

dataset = TimeSeriesDataset(df['value'].values)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_ds, val_ds, test_ds = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)


## Transformer Model

In [ ]:

import torch.nn as nn

class TransformerTS(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = nn.Linear(1, 64)
        encoder_layer = nn.TransformerEncoderLayer(d_model=64, nhead=4, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)
        self.fc = nn.Linear(64, 1)

    def forward(self, x):
        x = self.embed(x)
        x = self.transformer(x)
        return self.fc(x[:, -1, :])

model = TransformerTS()
model


## Training

In [ ]:

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()
EPOCHS = 15

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for x, y in train_loader:
        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred, y.squeeze())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss/len(train_loader):.4f}")


## Evaluation

In [ ]:

from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

model.eval()
y_true, y_pred = [], []

for x, y in DataLoader(test_ds, batch_size=64):
    with torch.no_grad():
        pred = model(x)
    y_true.extend(y.numpy())
    y_pred.extend(pred.numpy())

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae = mean_absolute_error(y_true, y_pred)
rmse, mae


## ARIMA Baseline

In [ ]:

from statsmodels.tsa.arima.model import ARIMA

train_series = df['value'][:2000]
test_series = df['value'][2000:]

model_arima = ARIMA(train_series, order=(5,1,0))
fit = model_arima.fit()
forecast = fit.forecast(len(test_series))

rmse_arima = np.sqrt(mean_squared_error(test_series, forecast))
rmse_arima
